In [2]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """
    Walk upwards from `start` (or CWD) until we find a repo marker.
    Works in notebooks where __file__ is not available.
    """
    start = (start or Path.cwd()).resolve()
    markers = ("pyproject.toml", ".git", "setup.cfg", "requirements.txt")
    for p in (start, *start.parents):
        if any((p / m).exists() for m in markers):
            return p
    # last resort: assume current folder is repo root
    return start

REPO_ROOT = find_repo_root()
OUT_DIR = REPO_ROOT / "data" / "signals"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", Path.cwd())
print("REPO_ROOT:", REPO_ROOT)
print("OUT_DIR:", OUT_DIR)

CWD: r:\scanner\books
REPO_ROOT: C:\Users\MaartenEnde\Repos\scanner
OUT_DIR: C:\Users\MaartenEnde\Repos\scanner\data\signals


In [3]:
from scanner_bot.core.ports import BarStore
from scanner_bot.infra.db.sqllite_barstore import SQLiteBarStore

DB_PATH = Path(REPO_ROOT / "data/db/scanner.db")

store = SQLiteBarStore(DB_PATH)

print(store.list_bars('AAPL', '1d', None, None, 100))




[Bar(date=datetime.date(1980, 12, 12), ticker='AAPL', interval='1d', open=0.1283479928970337, high=0.1289059966802597, low=0.1283479928970337, close=0.1283479928970337, adj_close=0.09838948398828506, volume=469033600.0), Bar(date=datetime.date(1980, 12, 15), ticker='AAPL', interval='1d', open=0.12221000343561172, high=0.12221000343561172, low=0.12165199965238571, close=0.12165199965238571, adj_close=0.09325641393661499, volume=175884800.0), Bar(date=datetime.date(1980, 12, 16), ticker='AAPL', interval='1d', open=0.1132809966802597, high=0.1132809966802597, low=0.11272300034761429, close=0.11272300034761429, adj_close=0.08641161024570465, volume=105728000.0), Bar(date=datetime.date(1980, 12, 17), ticker='AAPL', interval='1d', open=0.11551299691200256, high=0.11607100069522858, low=0.11551299691200256, close=0.11551299691200256, adj_close=0.0885503962635994, volume=86441600.0), Bar(date=datetime.date(1980, 12, 18), ticker='AAPL', interval='1d', open=0.11886200308799744, high=0.1194199994